# Packeges

In [ ]:
!pip -q install rdkit==2024.3.4
!pip -q install scikit-fingerprints
!pip -q install kora py3Dmol
!pip -q install kaleido
!pip -q install jupyter-dash
!pip -q install molplotly --no-deps
!pip -q install molvs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.1/33.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.0/357.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 15.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing 

In [ ]:
import pandas as pd
import numpy as np
import re
import kaleido
import molplotly
from rdkit import Chem, RDConfig
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools, Scaffolds
from rdkit.Chem import Descriptors, Descriptors3D
from rdkit.DataStructs.cDataStructs import ExplicitBitVect
import kora.install.rdkit
from skfp.fingerprints import MAPFingerprint
import plotly.express as px
import matplotlib.pyplot as plt
from rdkit.Chem import rdMolDescriptors
from molvs.standardize import Standardizer
from molvs.charge import Uncharger, Reionizer
from molvs.fragment import LargestFragmentChooser
from molvs.tautomer import TautomerCanonicalizer
from rdkit.Chem.rdmolops import GetFormalCharge, RemoveStereochemistry
from sklearn.preprocessing import StandardScaler
from rdkit.Chem import MACCSkeys, RDKFingerprint, Pharm2D
from rdkit.Chem.Pharm2D import Gobbi_Pharm2D, Generate
import sklearn
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import math
from math import pi
from IPython.display import display, SVG, HTML
import json
from scipy.spatial import distance
from jupyter_dash import JupyterDash
from itertools import combinations
from joblib import Parallel, delayed
from sklearn.manifold import trustworthiness
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import pairwise_distances
from google.colab.output import serve_kernel_port_as_iframe
import plotly.io as pio

# Funtion


In [ ]:
                        #Dataset settings
def chemical_multiverse(dataset=None, ID=None, smiles_column_name=None, target_activities=[None],

                        #Calculation settings
                        vPCA=True, t_SNE=True, MACCS=True, ECFP=True, radius=3, druglikeness_descriptors=True, MAP4=True, signaturizer_code=['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5'],

                        #Visualization settings
                        palette='hsv_r', size_point=12.0, size_point_representation='normal_desviation',   point_shape='circle',

                        #Color scale configuration
                        evaluation_metric='mpIC50_value', metric_name='mpIC50',

                        #t_SNE configuration
                        perplexity=33, n_iterations=1000,

                        #Parallelization
                        n_jobs=-1):

   """
   Function designed to visualize a chemical multiverse with multiple activities or properties in an interactive scatter plot for a compound set.

   Args:
      dataset (CSV, XLSX, TSV, JSON, XML): DataFrame of compounds provided by the user.

      ID (str): Column name containing the ID of each compound.

      smiles_column_name (str): Column name containing SMILES notation.

      target_activities (list): List with the names of columns containing target activity values.

      vPCA (bool): User-defined variable for applying PCA.

      t_SNE (bool): User-defined variable for applying t-SNE.

      MACCS (bool): User-defined variable for applying MACCS keys.

      ECFP (bool): User-defined variable for applying ECFP.

      radius (int): User-defined variable for ECFP radius 2 or 3.

      druglikeness_descriptors (bool): User-defined variable for applying the 6 drug-likeness descriptors.

      signaturizer_code (list): User-defined variable for applying signaturizer with 6 different codes.

      palette (str): User-defined variable for defining a continuous color palette.

      perplexity (int): User-defined variable for adjusting the proximity of points in t-SNE, depending on dataset size.

      size_point (float): User-defined variable for adjusting point size in the plot.

      size_point_representation (str): User-defined variable for adjusting point size in the plot.

      point_shape (str): User-defined variable for adjusting point shape in the plot.

      evaluation_metric (str): User-defined variable for adjusting point shape in the plot.

      metric_name (str): User-defined variable for adjusting point shape in the plot.

      n_iterations (int): Number of iterations for t-SNE.

      n_jobs(int): Number of cores to parallelize the process


   Returns:
    Up to 50 interactive plots.
    DataFrame with all calculations performed.
   """



   def pretreatment(smi:str)->str:

        """
        SMILES Preprocessing: from the SMILES column, filters by allowed atoms and applies the STD, LFC, UC, RI and TC functions.

        Args:
            smi (str): SMILES to preprocess; name of the column containing the SMILES.

        Returns:
            str: Preprocessed SMILES or the corresponding error message.
        """
        try:
            mol = Chem.MolFromSmiles(smi)
            if mol is None:
                return "Invalid SMILES"
            mol = STD(mol)
            mol = LFC(mol)
            allowed_elements = {"H","B","C","N","O","F","Si","P","S","Cl","Se","Br","I"}
            actual_elements = set([atom.GetSymbol() for atom in mol.GetAtoms()])
            if len(actual_elements-allowed_elements) == 0:
                mol = UC(mol)
                mol = RI(mol)
                mol = TC(mol)
                return Chem.MolToSmiles(mol)
            else:
                return "Disallowed elements in SMILES"
        except Exception as e:
            return f'Error: {str(e)}'

   def parallel_pretreatment(data_frame:pd.DataFrame, smiles_column_name, n_jobs=n_jobs)->pd.DataFrame:
       '''
       Parallelize the pretreatment of Smiles

       Arg:
          data_frame(pd.DataFrame): DataFrame containing SMILES
          smiles_column_name(str): Name of the column containing SMILES
          n_jobs(int): Number of cores to parallelize the process

       Returns:
          data_frame(pd.DataFrame): DataFrame with the new column
       '''

       if n_jobs == -1:
          import multiprocessing
          n_jobs = multiprocessing.cpu_count()

       with Parallel(n_jobs=n_jobs) as parallel:
            canonical_smiles = parallel(delayed(pretreatment)(smiles) for smiles in data_frame[smiles_column_name])

       data_frame['Canonical_Smiles'] = canonical_smiles
       data_frame = data_frame[~data_frame["Canonical_Smiles"].isin(["Invalid SMILES", "Disallowed elements in SMILES", "Error general"])].reset_index(drop=True)
       return data_frame


   def similarity_calc(smi1:str, smi2:str, method:str='tanimoto', fp_type:str='MACCS')->list:

        """
        Similarity calculation between two compounds based on their processed SMILES.

        Args:
            smi1 (str): SMILES of the first compound to compare.
            smi2 (str): SMILES of the second compound to compare.
            method (str): Comparison method to use. Can be 'tanimoto'
            fp_type (str): Type of fingerprint to use. Can be 'MACCS' or 'ECFP' with radius 2 or 3.

        Returns:
            similarity (float): Similarity coefficient between the compounds.
        """

        mol1 = Chem.MolFromSmiles(smi1)
        mol2 = Chem.MolFromSmiles(smi2)

        if fp_type == 'MACCS':
            fp1 = MACCSkeys.GenMACCSKeys(mol1)
            fp2 = MACCSkeys.GenMACCSKeys(mol2)
        elif fp_type == 'ECFP':
            fp1 = AllChem.GetMorganFingerprintAsBitVect(mol1, radius, nBits=2048)
            fp2 = AllChem.GetMorganFingerprintAsBitVect(mol2, radius, nBits=2048)

        if method == 'tanimoto':
            similarity = round(DataStructs.TanimotoSimilarity(fp1, fp2), 3)
        return similarity


   def numpy_to_bitvect(array)->ExplicitBitVect:

       bitvect = ExplicitBitVect(len(array))
       for i, value in enumerate(array):
           if value:
               bitvect.SetBit(i)
       return bitvect


   def map4_tanimoto_calculation(fps_bitvect):

       n = len(fps_bitvect)
       sim_matrix = np.zeros((n, n))
       for i in range(n):
           for j in range(i+1, n):
               sim_matrix[i, j] = DataStructs.TanimotoSimilarity(fps_bitvect[i], fps_bitvect[j])
               sim_matrix[j, i] = sim_matrix[i, j]
       return sim_matrix


   def calculate_all_similarities_blocks(fp_type:str='MACCS', method:str='tanimoto')->pd.DataFrame:
        '''
        Similarity calculations between all pair of compounds using block processing.

        Args:
            fp_type (str): Type of fingerprint to use. Can be 'MACCS', 'ECFP' and MAP4 with radius 2 or 3.
            method (str): Comparison method to use. Can be 'tanimoto'
            block_size (int): Number of compounds to process in each block.

        Returns:
            similarity_df (pd.DataFrame): DataFrame with the similarity between all pairs of compounds.
        '''
        smiles = input_df["Canonical_Smiles"].to_list()
        compounds_ids = input_df[ID].to_list()
        num_smiles = len(smiles)

        if num_smiles <= 1000:
          block_size = 500
        elif num_smiles <=10000:
          block_size = 500
        else:
          block_size = 1000

        matrix = np.zeros((num_smiles, num_smiles))

        for i_start in range(0, num_smiles, block_size):
            i_end = min(i_start + block_size, num_smiles)
            for j_start in range(0, num_smiles, block_size):
                j_end = min(j_start + block_size, num_smiles)
                block_pairs = [(i,j) for i in range(i_start, i_end) for j in range(j_start, j_end) if i < j]
                with Parallel(n_jobs=n_jobs) as parallel:
                    block_similarities = parallel(delayed(similarity_calc)(smiles[i], smiles[j], method=method, fp_type=fp_type) for i, j in block_pairs)

                #Map the similarities back to the matrix
                k=0
                for i,j in block_pairs:
                    matrix[i,j] = block_similarities[k]
                    matrix[j,i] = block_similarities[k]
                    k+=1

        np.fill_diagonal(matrix, 1.0)

        similarity_df = pd.DataFrame(matrix, index=compounds_ids, columns=compounds_ids)


        return similarity_df


   def evaluate_metric(data_frame:pd.DataFrame, evaluation_metric:str='norma_desviation', metric_name:str='mpIC50')->float:
        """
        Generate a custom metric based on columns from a DataFrame and stores the result in a new column

        Arg:
           data_frame (str): pd.DataFrame, name of the dataframe containing the columns requerid for the evaluation
           evaluation_metric (str): Mathematical spression defining how to combine the columns. Example: '(-log(Activity1) + Activity2 - 0.5 * Activity3)'
           metric_name (str): Name of the new column where the results will be stored and the names showed in th graphics

        Returns:
           data_frame (pd.DataFrame): DataFrame with the new column
        """

        columns_in_expression = re.findall(r'[A-Za-z_][A-Za-z_0-9]*',evaluation_metric)

        missing_columns = [col for col in columns_in_expression if col not in data_frame.columns and not col.isnumeric()]
        if missing_columns:
          raise ValueError(f"Error: The following columns are missing in the provided dataset: {', '.join(missing_columns)}")


        expression_to_evaluate = evaluation_metric
        for col in columns_in_expression:
           if col in data_frame.columns:
             expression_to_evaluate = expression_to_evaluate.replace(col, f'data_frame["{col}"]')

        try:
          data_frame[metric_name] = eval(expression_to_evaluate)
        except Exception as e:
          raise ValueError(f"Error: {e}")

        return data_frame

   def applying_pca(data, n_components=2):

     '''
     Apply PCA to the data and return the transformated data.

     '''
     pca = PCA(n_components=n_components)
     pca_results = pca.fit_transform(data)
     explained_variance = pca.explained_variance_ratio_
     return pca_results, explained_variance


   def applying_tsne(data, perplexity: int, n_iterations: int):

     tsne = TSNE(n_components=2, verbose=0, perplexity=perplexity)
     tsne_results = tsne.fit_transform(data)
     return tsne_results


   def configure_plot_layout(fig:plt, data_frame:pd.DataFrame, size_point_representation)->plt:

      """
      Plot layout configuration.

      Args:
          fig: plot to configure.
          data_frame: DataFrame containing the plot data.

      Returns:
          fig: Plot with configured layout.
      """

      fig.update_layout(
          title_font=dict(size=25),
          plot_bgcolor='white',
          paper_bgcolor='white',
          xaxis=dict(mirror=True, showgrid=False, showline=True, zeroline=False, linecolor='black', title_font=dict(size=30),tickfont=dict(size=25)),
          yaxis=dict(mirror=True, showgrid=False, showline=True, zeroline=False, linecolor='black', title_font=dict(size=30), tickfont=dict(size=25)),
          coloraxis_colorbar=dict(title=metric_name, title_side= 'bottom', orientation="h", xanchor='center', yanchor='bottom', thickness=15, len=0.6, y=-0.29, title_font=dict(size=30), tickfont=dict(size=18)))
      plt.figure(dpi=1000)
      fig.update_traces(marker=dict(size=data_frame[size_point_representation], symbol='circle', line=dict( width=2, color='black'), opacity=1))
      return fig


   def calculate_size_point (descriptor):
        '''
        Funtion designed to generate the normalized values needed to visualize them in the size point

        Arg:
        descriptor(str): Name of drug-likeness descriptors

        Returns:
        Narmalized value of the drug-likeness descriptors
        '''
        return 7+np.log1p((descriptor - descriptor.min()) / (descriptor.max() - descriptor.min()))*12





   def calculate_similarity_correlation(original_space:pd.DataFrame, reduced_space:pd.DataFrame, metric='euclidean'):

        '''
        Calculates the correlation of pairwise similarities between the original and reduced spaces.

        Args:
          original_space: pd.DataFrame of shape (n_compounds, n_features)
          reduced_space: pd.DataFrame (n_compounds, n_components)
          metric(str): The distance metric to compute pairwise distances.

        Returns:
          correlation(float): The correlation between the original and reduced spaces. Values close to 1 indicare a strong preservation of relationships in the reduced space.

        '''

        original_distances = pairwise_distances(original_space, metric=metric)
        reduced_distances = pairwise_distances(reduced_space, metric=metric)

        original_flat = original_distances[np.triu_indices_from(original_distances, k=1)]
        reduced_flat = reduced_distances[np.triu_indices_from(reduced_distances, k=1)]

        correlation,_= pearsonr(original_flat, reduced_flat)

        return correlation



   def signaturizer_calculation(smiles_list:pd.DataFrame, signaturizer_code:list)->pd.DataFrame:

        """
        Calculation of the chemical signature from a list of SMILES.

        Args:
            smiles_list (df): List of SMILES to process.
            signaturizer_code (list): Selected chemical signature code for calculation.

        Returns:
            signature_df: DataFrame containing the calculated chemical signature.
        """

        signaturizer_mapping = {'A1': CCSpace.A1, 'A2': CCSpace.A2, 'A3': CCSpace.A3, 'A4': CCSpace.A4, 'A5': CCSpace.A5,
                                'B1': CCSpace.B1, 'B2': CCSpace.B2, 'B3': CCSpace.B3, 'B4': CCSpace.B4, 'B5': CCSpace.B5,
                                'C1': CCSpace.C1, 'C2': CCSpace.C2, 'C3': CCSpace.C3, 'C4': CCSpace.C4, 'C5': CCSpace.C5,
                                'D1': CCSpace.B1, 'D2': CCSpace.B2, 'D3': CCSpace.B3, 'D4': CCSpace.B4, 'D5': CCSpace.B5,
                                'E1': CCSpace.E1, 'E2': CCSpace.E2, 'E3': CCSpace.E3, 'E4': CCSpace.E4, 'E5': CCSpace.E5}
        if signaturizer_code not in signaturizer_mapping:
            raise ValueError(f"Invalid signaturizer code: {signaturizer_code}")
        signaturizer = Signaturizer(signaturizer_mapping[signaturizer_code])
        signature = signaturizer.infer_from_smiles(smiles_list)
        signature_df = pd.DataFrame(signature)
        return signature_df


   if signaturizer_code is None:
      signaturizer_code =[]

   signaturizer_name = {'A1':'2D Fingerprints', 'A2':'3D Fingerprints', 'A3':'Scaffolds', 'A4':'Structural keys', 'A5':'Physicochemistry',
                      'B1':'Mechanisms of action', 'B2':'Metabolic genes', 'B3':'Crystals', 'B4':'Binding', 'B5':'HTS Bioassays',
                      'C1':'Small molecule roles', 'C2':'Small molecule pathways', 'C3':'Signaling pathways', 'C4':'Biological processes', 'C5':'Interactome',
                      'D1':'Transcription', 'D2':'Cancer cell lines', 'D3':'Chemical genetics', 'D4':'Morphology', 'D5':'Cell Bioassays',
                      'E1':'Therapeutic areas', 'E2':'Indications', 'E3':'Side Effects', 'E4':'Diseases and Toxicology', 'E5':'Drug-Drug Interactions'}

   # Verification dataset is None
   if dataset is None:
      raise ValueError("The file name must be specified.")

   # Verification of the input dataset format
   file_readers = {

    'csv': pd.read_csv,
    'xlsx': pd.read_excel,
    'tsv': lambda f: pd.read_csv(f, delimiter='\t'),
    'json': pd.read_json,
    'xml': pd.read_xml
    }

   ext = dataset.split('.')[-1].lower()
   if ext not in file_readers:
      raise ValueError("Unsupported file format")
   # Read the dataset
   input_df = file_readers[ext](dataset)

   # Validation of required columns in the Dataframe
   if ID not in input_df.columns:
    raise ValueError(f"Error: The column {ID}, is missing in the provided  dataset")

   if smiles_column_name not in input_df.columns:
      raise ValueError(f"Error: The column {smiles_column_name}, is missing in the provided  dataset")

   missing_columns = set(target_activities) - set(input_df.columns)
   if missing_columns:
      raise ValueError(f"Error: The following target activities are missing in the provided dataset: {', '.join(missing_columns)}")

   # Validate types of variables
   if not isinstance(vPCA, bool):
      raise ValueError("Error: vPCA should be a boolean value (True or False)")

   if not isinstance(t_SNE, bool):
      raise ValueError("Error: t-SNE should be a boolean value (True or False)")

   if not isinstance(MACCS, bool):
      raise ValueError("Error: MACCS should be a boolean value (True or False)")

   if not isinstance(ECFP, bool):
      raise ValueError("Error: ECFP should be a boolean value (True or False)")

   if not isinstance(radius, int) or radius not in [2, 3]:
      raise ValueError("Error: radius should be an integer value of 2 or 3")

   if not isinstance(druglikeness_descriptors, bool):
      raise ValueError("Error: druglikeness_descriptors should be a boolean value (True or False)")

   if not isinstance(perplexity, int) or perplexity <= 0:
      raise ValueError("Error: perplexity should be an positive integer value")

   valid_signaturizer_codes = ['A1', 'A2', 'A3', 'A4', 'A5', 'B1', 'B2', 'B3', 'B4', 'B5', 'C1', 'C2', 'C3', 'C4', 'C5', 'D1', 'D2', 'D3', 'D4', 'D5', 'E1', 'E2', 'E3', 'E4', 'E5']

   if not all(code in valid_signaturizer_codes for code in signaturizer_code):
    invalid_codes = [code for code in signaturizer_code if code not in valid_signaturizer_codes]
    raise ValueError(f"Error: Invalid signaturizer code(s): {', '.join(invalid_codes)}")

   if not isinstance(size_point, float) or size_point <= 0:
      raise ValueError("Error: size_point should be a positive float value")

   if not isinstance(point_shape, str):
      raise ValueError("Error: point_shape should be a string value")

   # It always necessary at least one dimensionality reduction technique

   if not vPCA and not t_SNE:
       raise Warning('Warning: Both PCA and t-SNE are disable. You May want to enable at least one for similarity calculations')

   # Check for null values in key columns
   if input_df[smiles_column_name].isnull().any():
    raise ValueError("Error: The SMILES column contains null values. Please clean the data")

   if input_df[ID].isnull().any():
    raise ValueError("Error: The ID column contains null values. Pleas clean the data")

   # Check that ID is unique
   if not input_df[ID].is_unique:
    raise ValueError("Error: The ID column contains duplicate values. Please clean the data")

   # Validate target activities: check if values are numeric and non-null
   for activity in target_activities:
    if input_df[activity].isnull().any():
      raise ValueError(f'Error: The {activity} column contains null values. Please clean the data')
    if not pd.api.types.is_numeric_dtype(input_df[activity]):
      raise TypeError(f'Error: The {activity} column contains non-numeric values. Please ensure all values are numeric')


   # Definition of functions to processing SMILES
   STD = Standardizer()
   LFC = LargestFragmentChooser()
   UC = Uncharger()
   RI = Reionizer()
   TC = TautomerCanonicalizer()

   Magenta_color = '\033[95m'
   Default_color = '\033[0m'

   # Prossesing of Smiles and evaluation of activities
   #input_df = parallel_pretreatment(data_frame=input_df, smiles_column_name=smiles_column_name)
   #input_df = input_df.drop_duplicates(subset=['Canonical_Smiles'], keep='first').reset_index(drop=True)
   input_df['mpIC50_value'] = -np.log10(input_df[target_activities]).mean(axis=1)
   max_value = input_df['mpIC50_value'].max()
   min_value = input_df['mpIC50_value'].min()
   input_df['std'] = input_df[target_activities].std(axis=1)
   input_df['norma'] = ((input_df['std'] - input_df['std'].min()) / (input_df['std'].max() - input_df['std'].min()))
   input_df['normal_desviation'] = input_df['norma'] * (size_point-10)+9
   input_df['normal_desviation'] = input_df['normal_desviation'].fillna(10)

   input_df = input_df.rename(columns={'Smiles': 'Canonical_Smiles'})

   # Calcualtion of descriptors
   smiles_list = input_df[[ID,"Canonical_Smiles"]]
   smiles_mols = [Chem.MolFromSmiles(x) for x in smiles_list["Canonical_Smiles"]]
   descriptors_df = pd.DataFrame({
            "HBA": [Descriptors.NumHAcceptors(mol) for mol in smiles_mols],
            "MW": [Descriptors.MolWt(mol) for mol in smiles_mols],
            "HBD": [Descriptors.NumHDonors(mol) for mol in smiles_mols],
            "RB": [Descriptors.NumRotatableBonds(mol) for mol in smiles_mols],
            "LogP": [Descriptors.MolLogP(mol) for mol in smiles_mols],
            "TPSA": [Descriptors.TPSA(mol) for mol in smiles_mols]
        }, index=smiles_list.index)

   descriptors_df['Canonical_Smiles'] = smiles_list['Canonical_Smiles']
   input_df = pd.merge(input_df, descriptors_df, on='Canonical_Smiles', how='inner')


   for descriptor in ['HBA', 'LogP', 'TPSA', 'MW', 'HBD', 'RB']:
    input_df[f'size_point_{descriptor}'] = calculate_size_point(descriptors_df[descriptor])

   input_df = evaluate_metric(input_df, evaluation_metric=evaluation_metric, metric_name=metric_name)

   pd.DataFrame(input_df).to_csv('input_df.csv')

   # If MACCS keys are selected by the user, this section will build a matrix to calculate the similarity
   if MACCS is True:

              matrix_MACCS_df = calculate_all_similarities_blocks(fp_type='MACCS', method='tanimoto')
              data_MACCS = matrix_MACCS_df.iloc[:,0:].values

              # If user select PCA, this section will recover the values of similarity and train the model to calcualte principal componentes
              if vPCA is True:

                    pca_results_MACCS, exp_var_pca_maccs = applying_pca(data_MACCS)

                    data_MACCS_PCA = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'component1':pca_results_MACCS[:,0], 'component2':pca_results_MACCS[:,1]})

                    loadings_maccs = pca_results_MACCS[:,:2]**2
                    loadings_maccs = loadings_maccs / loadings_maccs.sum(axis=0)

                    dataset_MACCS_PCA= pd.merge(data_MACCS_PCA, input_df, on=("Canonical_Smiles", ID), how='inner')
                    dataset_MACCS_PCA[['PC1 contribution', 'PC2 contribution']] = loadings_maccs

                    # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
                    trust_pca_maccs = trustworthiness(data_MACCS, pca_results_MACCS)
                    correlation_pca_maccs = (calculate_similarity_correlation(data_MACCS, pca_results_MACCS))

                    print(f'Trustworthiness: {Magenta_color}{trust_pca_maccs:3f}{Default_color}')
                    print(f'Correlation: {Magenta_color}{correlation_pca_maccs:3f}{Default_color}')

                    dataset_MACCS_PCA.to_csv('dataset_MACCS_PCA.csv')
                    x_label = f'component1 {exp_var_pca_maccs[0]}'
                    y_label = f'component2 {exp_var_pca_maccs[1]}'

                    # This section creates and customizes a scatter plot, generating an interactive plot and producing a downloadable file with higher resolution.
                    fig_pca = px.scatter(dataset_MACCS_PCA, x='component1', y='component2', color=metric_name, color_continuous_scale=palette, title='PCA - MACCS',
                      labels={'component1': f'Component 1 ({exp_var_pca_maccs[0]*100:.3}%)', 'component2': f'Component 2 ({exp_var_pca_maccs[1]*100:.3}%)'},
                      width=1200,
                      height=900,
                      #range_color=[min_value, max_value]
                                         )
                    fig_pca = configure_plot_layout(fig_pca, data_frame=dataset_MACCS_PCA, size_point_representation=size_point_representation)
                    pio.write_image(fig_pca, 'fig_PCA_MACCS.png')
                    app_maker_maccs_pca = molplotly.add_molecules(fig=fig_pca, df=dataset_MACCS_PCA, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'PC1 contribution', 'PC2 contribution'])
                    serve_kernel_port_as_iframe("localhost")
                    app_maker_maccs_pca.run(port=8060)
                    #fig_pca.show()



              # "If the user sets the variable t_SNE to true, this section will retrieve similarity calculation values to train the model. It is necessary to define the perplexity value according to the dataset size
              if t_SNE is True:

                      tsne_results_MACCS = applying_tsne(data_MACCS, perplexity=perplexity, n_iterations=n_iterations)

                      all_dataset_tSNE = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles':input_df['Canonical_Smiles'], 'axis 1':tsne_results_MACCS[:,0], 'axis 2':tsne_results_MACCS[:,1]})

                      all_dataset_tSNE = pd.merge(all_dataset_tSNE, input_df, on=("Canonical_Smiles", ID), how='inner')

                      all_dataset_tSNE.to_csv('dataset_tSNE_MACCS.csv')

                      # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and tsne resulta
                      trust_tsne_maccs = trustworthiness(data_MACCS, tsne_results_MACCS)
                      correlation_tsne_maccs = (calculate_similarity_correlation(data_MACCS, tsne_results_MACCS))
                      print(f'Trustworthiness: {Magenta_color}{trust_tsne_maccs:3f}{Default_color}')
                      print(f'Correlation: {Magenta_color}{correlation_tsne_maccs:3f}{Default_color}')


                      # Plot construction
                      fig_tsne_MACCS = px.scatter(all_dataset_tSNE, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title='T-SNE - MACCS',
                      labels={'axis 1': 'Component 1', 'axis 2': 'Component 2'},
                      width=1200,
                      height=900,
                      #range_color=[min_value, max_value]
                                                  )
                      fig_tsne_MACCS = configure_plot_layout(fig_tsne_MACCS, data_frame=all_dataset_tSNE, size_point_representation=size_point_representation)
                      pio.write_image(fig_tsne_MACCS, 'fig_tsne_MACCS.png')
                      app_maker_maccs_tsne = molplotly.add_molecules(fig=fig_tsne_MACCS, df=all_dataset_tSNE, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                      #fig_tsne_MACCS.show()
                      serve_kernel_port_as_iframe("localhost")
                      app_maker_maccs_tsne.run(port=8062)



  # If the user sets ECFP to true, this section will retrieve similarity_calc to generate a similarity matrix using the Extended Connectivity Fingerprint(ECFP). It is necessary to specify the radius to calculate either ECFP4 or ECFP6
   if ECFP is True:

               matrix_ECFP_df = calculate_all_similarities_blocks(fp_type='ECFP', method='tanimoto')
               data_ECFP = matrix_ECFP_df.iloc[:,0:].values

                # This section performs PCA model training and calculates the principal components.
               if vPCA is True:

                      pca_results_ECFP, exp_var_pca_ecfp = applying_pca(data_ECFP)

                      data_ECFP_PCA = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'component1':pca_results_ECFP[:,0], 'component2':pca_results_ECFP[:,1]})

                      loadings_ecfp = pca_results_ECFP[:,:2]**2
                      loadings_ecfp = loadings_ecfp / loadings_ecfp.sum(axis=0)

                      dataset_ECFP_PCA= pd.merge(data_ECFP_PCA, input_df, on=("Canonical_Smiles", ID), how='inner')
                      dataset_ECFP_PCA[['PC1 contribution', 'PC2 contribution']] = loadings_ecfp


                      # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
                      trust_pca_ecfp = trustworthiness(data_ECFP, pca_results_ECFP)
                      correlation_pca_ecfp = (calculate_similarity_correlation(data_ECFP, pca_results_ECFP))

                      print(f'Trustworthiness: {Magenta_color}{trust_pca_ecfp:3f}{Default_color}')
                      print(f'Correlation: {Magenta_color}{correlation_pca_ecfp:3f}{Default_color}')


                      dataset_ECFP_PCA.to_csv('dataset_ECFP_PCA.csv')
                      x_label = f'component1 {exp_var_pca_ecfp[0]}'
                      y_label = f'component2 {exp_var_pca_ecfp[1]}'

                      # Plot configuration
                      fig_pca_ECFP = px.scatter(dataset_ECFP_PCA, x='component1', y='component2', color=metric_name, color_continuous_scale=palette, title='PCA - ECFP',
                       labels={'component1': f'Component 1 ({exp_var_pca_ecfp[0]*100:.3}%)', 'component2': f'Component 2 ({exp_var_pca_ecfp[1]*100:.3}%)'},
                       width=1200,
                       height=900,
                       #range_color=[min_value, max_value]
                                                )
                      fig_pca_ECFP = configure_plot_layout(fig_pca_ECFP, data_frame=dataset_ECFP_PCA, size_point_representation=size_point_representation)
                      pio.write_image( fig_pca_ECFP, 'fig_PCA_ECFP.png')
                      app_maker_ecfp_pca = molplotly.add_molecules(fig=fig_pca_ECFP, df=dataset_ECFP_PCA, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'PC1 contribution', 'PC2 contribution'])
                      #fig_pca_ECFP.show()
                      serve_kernel_port_as_iframe("localhost")
                      app_maker_ecfp_pca.run(port=8061)



                # t-SNE technique application with ECFP similarity calculation
               if t_SNE is True:

                      tsne_results_ECFP = applying_tsne(data_ECFP, perplexity=perplexity, n_iterations=n_iterations)

                      dataset_tSNE_ECFP = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'axis 1':tsne_results_ECFP[:,0], 'axis 2':tsne_results_ECFP[:,1]})

                      dataset_tSNE_ECFP = pd.merge(dataset_tSNE_ECFP, input_df, on=("Canonical_Smiles", ID), how='inner')

                      dataset_tSNE_ECFP.to_csv('dataset_tSNE_ECFP.csv')

                      # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
                      trust_tsne_ecfp = trustworthiness(data_ECFP, tsne_results_ECFP)
                      correlation_tsne_ecfp = (calculate_similarity_correlation(data_ECFP, tsne_results_ECFP))

                      print(f'Trustworthiness: {Magenta_color}{trust_tsne_ecfp:3f}{Default_color}')
                      print(f'Correlation: {Magenta_color}{correlation_tsne_ecfp:3f}{Default_color}')

                      # Plot configuration
                      fig_tsne_ECFP = px.scatter(dataset_tSNE_ECFP, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title='T-SNE - ECFP', labels={'axis 1': 'Component 1', 'axis 2': 'Component 2'},
                       width=1200,
                       height=900,
                       #range_color=[min_value, max_value]
                                                 )
                      fig_tsne_ECFP = configure_plot_layout(fig_tsne_ECFP, data_frame=dataset_tSNE_ECFP, size_point_representation=size_point_representation)
                      pio.write_image( fig_tsne_ECFP, 'fig_tsne_ECFP.png')
                      app_maker_ecfp_tsne = molplotly.add_molecules(fig=fig_tsne_ECFP, df=dataset_tSNE_ECFP, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                      #fig_tsne_ECFP.show()
                      serve_kernel_port_as_iframe("localhost")
                      app_maker_ecfp_tsne.run(port=8064)



   # This section calcualte Hidrogen Bond Aceptor(HBA), Hidrogen Bond Donator(HBD), Partitipon Coeficient(LogP), Total Polar Surface Area(TPSA), Molecular Weight(MW), Rotable Bonds(RB) descriptors form the processed SMILES
   if druglikeness_descriptors is True:
          smiles_list = pd.merge(smiles_list, descriptors_df, on='Canonical_Smiles', how='inner')
          descriptors_values = smiles_list.iloc[:,2:6].values
          molecular_descriptors_data_std = StandardScaler().fit_transform(descriptors_values) #Scaling of descriptor values
          df_MD = pd.DataFrame(molecular_descriptors_data_std)

          # Recover the DataFrame with scaled descriptor values to train the model and combine the ID, SMILES, and principal components into a single DataFrame.
          if vPCA is True:

              pca_results_druglikeness, explained_variance_druglikeness =  applying_pca(molecular_descriptors_data_std)

              dataset_druglikeness_pca = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'component1':pca_results_druglikeness[:,0], 'component2':pca_results_druglikeness[:,1]})

              loadings_druglikeness = pca_results_druglikeness[:,:2]**2
              loadings_druglikeness = loadings_druglikeness / loadings_druglikeness.sum(axis=0)

              dataset_druglikeness_pca[['PC1 contribution', 'PC2 contribution']] = loadings_druglikeness


              dataset_druglikeness_pca= pd.merge(dataset_druglikeness_pca, input_df, on=("Canonical_Smiles", ID), how='inner')

              # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
              trust_pca_druglikeness = trustworthiness(molecular_descriptors_data_std, pca_results_druglikeness)
              correlation_pca_druglikeness = (calculate_similarity_correlation(molecular_descriptors_data_std, pca_results_druglikeness))
              print(f'Trustworthiness: {Magenta_color}{trust_pca_druglikeness:3f}{Default_color}')
              print(f'Correlation: {Magenta_color}{correlation_pca_druglikeness:3f}{Default_color}')

              dataset_druglikeness_pca.to_csv('dataset_druglikeness_PCA.csv')
              x_label = f'component1 {explained_variance_druglikeness[0]}'
              y_label = f'component2 {explained_variance_druglikeness[1]}'

              # Plot configuration
              fig_pca_druglikeness = px.scatter(dataset_druglikeness_pca, x='component1', y='component2', color=metric_name, color_continuous_scale=palette, title='PCA - Drug-likeness descriptors', labels={'component1': f'Component 1 ({explained_variance_druglikeness[0]*100:.3}%)', 'component2': f'Component 2 ({explained_variance_druglikeness[1]*100:.3}%)'},
                 width=1200,
                 height=900
                 #range_color=[min_value, max_value]
                                                )
              fig_pca_druglikeness = configure_plot_layout(fig_pca_druglikeness, data_frame=dataset_druglikeness_pca, size_point_representation=size_point_representation)
              pio.write_image( fig_pca_druglikeness, 'fig_pca_druglikeness.png')
              app_maker = molplotly.add_molecules(fig= fig_pca_druglikeness, df=dataset_druglikeness_pca, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'HBA', 'LogP', 'TPSA', 'MW', 'HBD', 'RB', 'PC1 contribution', 'PC2 contribution'])
              #fig_pca_MD.show()
              serve_kernel_port_as_iframe("localhost")
              app_maker.run(port=8066)


          # run the t_SNE model with scaled descriptors values
          if t_SNE is True:

                tsne_results_MD = applying_tsne(molecular_descriptors_data_std, perplexity=perplexity, n_iterations=n_iterations)

                dataset_tSNE_MD = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'axis 1':tsne_results_MD[:,0], 'axis 2':tsne_results_MD[:,1]})


                dataset_tSNE_MD = pd.merge(dataset_tSNE_MD, input_df, on=("Canonical_Smiles", ID), how='inner')


                dataset_tSNE_MD.to_csv('dataset_druglikeness_tSNE.csv')

                # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
                trust_tsne_druglikeness = trustworthiness(molecular_descriptors_data_std, tsne_results_MD)
                correlation_tsne_druglikeness = (calculate_similarity_correlation(molecular_descriptors_data_std, tsne_results_MD))
                print(f'Trustworthiness: {Magenta_color}{trust_tsne_druglikeness:3f}{Default_color}')
                print(f'Correlation: {Magenta_color}{correlation_tsne_druglikeness:3f}{Default_color}')

                # Plot configuration
                fig_tsne_druglikeness = px.scatter(dataset_tSNE_MD, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title='T-SNE - Drug-likeness descriptors', labels={'axis 1': 'Component 1', 'axis 2': 'Component 2'},
                  width=1200,
                  height=900,
                  #range_color=[min_value, max_value]
                                                   )
                fig_tsne_druglikeness = configure_plot_layout(fig_tsne_druglikeness, data_frame=dataset_tSNE_MD, size_point_representation=size_point_representation)
                pio.write_image(fig_tsne_druglikeness, 'fig_tsne_druglikeness.png')
                app_maker = molplotly.add_molecules(fig=fig_tsne_druglikeness, df=dataset_tSNE_MD, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                #fig_tsne_MD.show()
                serve_kernel_port_as_iframe("localhost")
                app_maker.run(port=8068)



   # If user decicde to obtein bioactive descriptors, this section will recovery the signaturizer_calcualtion funtion to build a dataframe with the values of the signaturizes selectioned
   if signaturizer_code is not None:
    !pip install signaturizer3d --no-deps
    from signaturizer3d import Signaturizer, CCSpace

    for code in signaturizer_code:
      smiles = input_df[[ID,'Canonical_Smiles']]
      signature_df = signaturizer_calculation(smiles_list=smiles['Canonical_Smiles'], signaturizer_code = code)


      # With signaturizer fingerprint we train PCA model
      if vPCA is True:

              pca_results_signatures, explained_variance_signatures = applying_pca(signature_df)

              dataset_signatures_pca = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'component1':pca_results_signatures[:,0], 'component2':pca_results_signatures[:,1]})

              loadings_signatures = pca_results_signatures[:,:2]**2
              loadings_signatures = loadings_signatures / loadings_signatures.sum(axis=0)

              dataset_signatures_pca[['PC1 contribution', 'PC2 contribution']] = loadings_signatures

              dataset_signatures_pca= pd.merge(dataset_signatures_pca, input_df, on=("Canonical_Smiles", ID), how='inner')

              dataset_signatures_pca[['PC1 contribution', 'PC2 contribution']] = loadings_signatures

              # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
              trust_pca_signatures = trustworthiness(signature_df, pca_results_signatures)
              correlation_pca_signatures = (calculate_similarity_correlation(signature_df, pca_results_signatures))

              print(f'Trustworthiness: {Magenta_color}{trust_pca_signatures:3f}{Default_color}')
              print(f'Correlation: {Magenta_color}{correlation_pca_signatures:3f}{Default_color}')

              dataset_signatures_pca.to_csv(f'dataset_{code}_PCA.csv')
              x_label = f'component1 {explained_variance_signatures[0]}'
              y_label = f'component2 {explained_variance_signatures[1]}'

              # Plot configuration
              fig_pca_mechanisim = px.scatter(dataset_signatures_pca, x='component1',y='component2', color=metric_name, color_continuous_scale=palette, title=(f'PCA - {signaturizer_name.get(code)}'), labels={'component1': f'Component 1 ({explained_variance_signatures[0]*100:.3}%)', 'component2': f'Component 2 ({explained_variance_signatures[1]*100:.3}%)'},
               width=1200,
               height=900,
               #range_color=[min_value, max_value]
                                              )
              fig_pca_mechanisim = configure_plot_layout(fig_pca_mechanisim, data_frame=dataset_signatures_pca, size_point_representation=size_point_representation)
              pio.write_image(fig_pca_mechanisim, f'fig_pca_{signaturizer_name.get(code)}.png')
              app_maker = molplotly.add_molecules(fig=fig_pca_mechanisim, df=dataset_signatures_pca, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'PC1 contribution', 'PC2 contribution'])
              #fig_pca_mechanisim.show()
              serve_kernel_port_as_iframe("localhost")
              app_maker.run(port=8070)

      # Run t_SNE model to signaturizer fingerprint
      if t_SNE is True:

                tsne_results_signatures = applying_tsne(signature_df, n_iterations=n_iterations, perplexity=perplexity)

                dataset_tSNE_signatures = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'axis 1':tsne_results_signatures[:,0], 'axis 2':tsne_results_signatures[:,1]})

                dataset_tSNE_signatures = pd.merge(dataset_tSNE_signatures, input_df, on=("Canonical_Smiles", ID), how='inner')

                dataset_tSNE_signatures.to_csv(f'dataset_{code}_tSNE.csv')

                # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
                trust_tsne_signatures = trustworthiness(signature_df, tsne_results_signatures)
                correlation_tsne_signatures = (calculate_similarity_correlation(signature_df, tsne_results_signatures))
                print(f'Trustworthiness: {Magenta_color}{trust_tsne_signatures:3f}{Default_color}')
                print(f'Correlation: {Magenta_color}{correlation_tsne_signatures:3f}{Default_color}')

                # Plot config
                fig_tsne_signaturizer = px.scatter(dataset_tSNE_signatures, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title=(f't-SNE - {signaturizer_name.get(code)}'), labels={'axis 1': 'Component 1', 'axis 2': 'Component 2'},
                    width=1200,
                    height=900,
                    #range_color=[min_value, max_value]
                                                   )
                fig_tsne_signaturizer = configure_plot_layout(fig_tsne_signaturizer, data_frame=dataset_tSNE_signatures, size_point_representation=size_point_representation)
                pio.write_image(fig_tsne_signaturizer, f'fig_tsne_{signaturizer_name.get(code)}.png')
                app_maker = molplotly.add_molecules(fig=fig_tsne_signaturizer, df=dataset_tSNE_signatures, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                #fig_tsne_signaturizer.show()
                serve_kernel_port_as_iframe("localhost")
                app_maker.run(port=8072)

   if MAP4 is True:

               smiles_list = input_df["Canonical_Smiles"].tolist()
               map_4 = MAPFingerprint()
               fingerprints_map4 = map_4.transform(smiles_list)
               fps_bitvect = [numpy_to_bitvect(fp) for fp in fingerprints_map4]
               data_MAP4 = map4_tanimoto_calculation(fps_bitvect)


              # This section performs PCA model training and calculates the principal components.
               if vPCA is True:

                      pca_results_MAP4, exp_var_pca_MAP4 = applying_pca(data_MAP4)

                      data_MAP4_PCA = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'component1':pca_results_MAP4[:,0], 'component2':pca_results_MAP4[:,1]})

                      loadings_MAP4 = pca_results_MAP4[:, :2]**2
                      loadings_MAP4 = loadings_MAP4 / loadings_MAP4.sum(axis=0)

                      dataset_MAP4_PCA= pd.merge(data_MAP4_PCA, input_df, on=("Canonical_Smiles", ID), how='inner')
                      dataset_MAP4_PCA = dataset_MAP4_PCA.iloc[:loadings_MAP4.shape[0]]
                      dataset_MAP4_PCA.to_csv('dataset_MAP4_PCA.csv')
                      dataset_MAP4_PCA[['PC1 contribution', 'PC2 contribution']] = loadings_MAP4



                      # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
                      trust_pca_MAP4 = trustworthiness(data_MAP4, pca_results_MAP4)
                      correlation_pca_MAP4 = (calculate_similarity_correlation(data_MAP4, pca_results_MAP4))
                      print(f'Trustworthiness: {Magenta_color}{trust_pca_MAP4:3f}{Default_color}')
                      print(f'Correlation: {Magenta_color}{correlation_pca_MAP4:3f}{Default_color}')


                      dataset_MAP4_PCA.to_csv('dataset_MAP4_PCA.csv')
                      x_label = f'component1 {exp_var_pca_MAP4[0]}'
                      y_label = f'component2 {exp_var_pca_MAP4[1]}'

                      # Plot configuration
                      fig_pca_MAP4 = px.scatter(dataset_MAP4_PCA, x='component1', y='component2', color=metric_name, color_continuous_scale=palette, title='PCA - MAP4',
                       labels={'component1': f'Component 1 ({exp_var_pca_MAP4[0]*100:.3}%)', 'component2': f'Component 2 ({exp_var_pca_MAP4[1]*100:.3}%)'},
                       width=1200,
                       height=900,
                       #range_color=[min_value, max_value]
                                                )
                      fig_pca_MAP4 = configure_plot_layout(fig_pca_MAP4, data_frame=dataset_MAP4_PCA, size_point_representation=size_point_representation)
                      pio.write_image( fig_pca_MAP4, 'fig_PCA_MAP4.png')
                      app_maker_MAP4_pca = molplotly.add_molecules(fig=fig_pca_MAP4, df=dataset_MAP4_PCA, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles', 'PC1 contribution', 'PC2 contribution'])
                      #fig_pca_ECFP.show()
                      serve_kernel_port_as_iframe("localhost")
                      app_maker_MAP4_pca.run(port=8061)



                # t-SNE technique application with ECFP similarity calculation
               if t_SNE is True:

                      tsne_results_MAP4 = applying_tsne(data_MAP4, perplexity=perplexity, n_iterations=n_iterations)

                      dataset_tSNE_MAP4 = pd.DataFrame({ID:input_df[ID], 'Canonical_Smiles': input_df['Canonical_Smiles'], 'axis 1':tsne_results_MAP4[:,0], 'axis 2':tsne_results_MAP4[:,1]})

                      dataset_tSNE_MAP4 = pd.merge(dataset_tSNE_MAP4, input_df, on=("Canonical_Smiles", ID), how='inner')

                      dataset_tSNE_MAP4.to_csv('dataset_tSNE_MAP4.csv')

                      # Calculate Trustworthiness and Correlation metrics using result of similarity calculation and pca results
                      trust_tsne_MAP4 = trustworthiness(data_MAP4, tsne_results_MAP4)
                      correlation_tsne_MAP4 = (calculate_similarity_correlation(data_MAP4, tsne_results_MAP4))
                      print(f'Trustworthiness: {Magenta_color}{trust_tsne_MAP4:3f}{Default_color}')
                      print(f'Correlation: {Magenta_color}{correlation_tsne_MAP4:3f}{Default_color}')

                      # Plot configuration
                      fig_tsne_MAP4 = px.scatter(dataset_tSNE_MAP4, x='axis 1', y='axis 2', color=metric_name, color_continuous_scale=palette, title='T-SNE - MAP4', labels={'axis 1': 'Component 1', 'axis 2': 'Component 2'},
                       width=1200,
                       height=900,
                       #range_color=[min_value, max_value]
                                                 )
                      fig_tsne_MAP4 = configure_plot_layout(fig_tsne_MAP4, data_frame=dataset_tSNE_MAP4, size_point_representation=size_point_representation)
                      pio.write_image( fig_tsne_MAP4, 'fig_tsne_MAP4.png')
                      app_maker_ecfp_MAP4 = molplotly.add_molecules(fig=fig_tsne_MAP4, df=dataset_tSNE_MAP4, smiles_col="Canonical_Smiles", title_col=ID, color_col=metric_name, caption_cols=['Canonical_Smiles'])
                      #fig_tsne_ECFP.show()
                      serve_kernel_port_as_iframe("localhost")
                      app_maker_ecfp_MAP4.run(port=8064)

# Chemical multiverse

In [ ]:
chemical_multiverse(dataset='/content/test.csv', smiles_column_name='Smiles', ID='ID', target_activities=['target_objects'], perplexity=5, n_jobs=-1, evaluation_metric='target_objects', metric_name='Targets', MACCS=False, ECFP=False, signaturizer_code=None)

/usr/local/lib/python3.11/dist-packages/pandas/core/internals/blocks.py:393: RuntimeWarning: divide by zero encountered in log10
  result = func(self.values, **kwargs)


Trustworthiness: 0.988497
Correlation: 0.999719


/usr/local/lib/python3.11/dist-packages/dash/dash.py:587: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<Figure size 6400x4800 with 0 Axes>

<Figure size 6400x4800 with 0 Axes>

<Figure size 6400x4800 with 0 Axes>

<Figure size 6400x4800 with 0 Axes>